# LIB

In [1]:
import os
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import datetime as dt

import lib.survival_f as surv

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Auxiliary functions

In [2]:
def stratify_pares(data, data_pop):
    '''
        Stratification countings from the matched pairs dataset and the eligible population of
        the specific vaccine.
        
        Args:
            data:
                pandas.DataFrame. File of pairs with only the basic dates and type of individuals 
                (case or control).
            data_pop:
                pandas.DataFrame. File of eligible individuals for the matching of the vaccine.
        Return:
            hashd:
                dictionary.
    '''
    # --> collect data on age and sex
    data = data.merge(data_pop[["cpf", "sexo", "idade", "bairro"]], left_on="CPF", right_on="cpf", how="left")
    cols = ["DATA D1", "DATA D2"]
    for j in cols:
        data[j] = pd.to_datetime(data[j], format="%Y-%m-%d", errors="coerce")
    data["VACINA STATUS ATE FIM DA COORTE"] = data.apply(lambda x: f_vacina(x, config["final cohort"]), axis=1)
    
    # --> Strata conditions
    only_pareados = data["PAREADO"]==True
    only_d1 = pd.notna(data["DATA D1"]) & pd.isna(data["DATA D2"])
    d1_and_d2 = pd.notna(data["DATA D1"]) & pd.notna(data["DATA D2"])
    no_vaccine = pd.isna(data["DATA D1"]) & pd.isna(data["DATA D2"])
    males = data["sexo"]=="M"
    females = data["sexo"]=="F"
    idade_6069 = (data["idade"]>=60) & (data["idade"]<=69)
    idade_7079 = (data["idade"]>=70) & (data["idade"]<=79)
    idade_80 = (data["idade"]>=80) & (data["idade"]<=105)
    tipo_caso = data["TIPO"]=="CASO"
    tipo_controle = data["TIPO"]=="CONTROLE"
    obito_covid = pd.notna(data["DATA OBITO COVID"])
    obito_geral = pd.notna(data["DATA OBITO GERAL"])
    novac = data["VACINA STATUS ATE FIM DA COORTE"]=="NO VACCINE"
    d1_013 = data["VACINA STATUS ATE FIM DA COORTE"]=="D1 0-13 DAYS"
    d1_14 = data["VACINA STATUS ATE FIM DA COORTE"]=="D1 >= 14 DAYS"
    d2_013 = data["VACINA STATUS ATE FIM DA COORTE"]=="D2 0-13 DAYS"
    d2_14 = data["VACINA STATUS ATE FIM DA COORTE"]=="D2 >= 14 DAYS"
    
    # --> Apply strata conditions for whole population and separately for case and control individuals.
    hashd = {
        "TOTAL": data[only_pareados],
        "CASO": data[only_pareados & tipo_caso],
        "CONTROLE": data[only_pareados & tipo_controle],
        "NO VACCINE": data[(only_pareados) & (no_vaccine)],
        "ONLY D1": data[(only_pareados) & (only_d1)],
        "D1+D2": data[(only_pareados) & (d1_and_d2)],
        "MALES": data[(only_pareados) & (males)],
        "MALES CASO": data[(only_pareados) & (males) & (tipo_caso)],
        "MALES CONTROLE": data[(only_pareados) & (males) & (tipo_controle)],
        "FEMALES": data[(only_pareados) & (females)],
        "FEMALES CASO": data[(only_pareados) & (females) & (tipo_caso)],
        "FEMALES CONTROLE": data[(only_pareados) & (females) & (tipo_controle)],
        "60-69": data[(only_pareados) & (idade_6069)],
        "60-69 CASO": data[(only_pareados) & (idade_6069) & (tipo_caso)],
        "60-69 CONTROLE": data[(only_pareados) & (idade_6069) & (tipo_controle)],
        "70-79": data[(only_pareados) & (idade_7079)],
        "70-79 CASO": data[(only_pareados) & (idade_7079) & (tipo_caso)],
        "70-79 CONTROLE": data[(only_pareados) & (idade_7079) & (tipo_controle)],
        "80+": data[(only_pareados) & (idade_80)],
        "80+ CASO": data[(only_pareados) & (idade_80) & (tipo_caso)],
        "80+ CONTROLE": data[(only_pareados) & (idade_80) & (tipo_controle)],
        "OBITO COVID TOTAL": data[(only_pareados) & (obito_covid)],
        "OBITO COVID CASO": data[(only_pareados) & (obito_covid) & (tipo_caso)],
        "OBITO COVID CONTROLE": data[(only_pareados) & (obito_covid) & (tipo_controle)],
        "OBITO GERAL TOTAL": data[(only_pareados) & (obito_geral)],
        "OBITO GERAL CASO": data[(only_pareados) & (obito_geral) & (tipo_caso)],
        "OBITO GERAL CONTROLE": data[(only_pareados) & (obito_geral) & (tipo_controle)],
        "SEM VACINA TOTAL": data[(only_pareados) & (novac)],
        "SEM VACINA CASO": data[(only_pareados) & (novac) & (tipo_caso)],
        "SEM VACINA CONTROLE": data[(only_pareados) & (novac) & (tipo_controle)],
        "D1 0-13 TOTAL": data[(only_pareados) & (d1_013)],
        "D1 0-13 CASO": data[(only_pareados) & (d1_013) & (tipo_caso)],
        "D1 0-13 CONTROLE": data[(only_pareados) & (d1_013) & (tipo_controle)],
        "D1 >=14 TOTAL": data[(only_pareados) & (d1_14)],
        "D1 >=14 CASO": data[(only_pareados) & (d1_14) & (tipo_caso)],
        "D1 >=14 CONTROLE": data[(only_pareados) & (d1_14) & (tipo_controle)],
        "D2 0-13 TOTAL": data[(only_pareados) & (d2_013)],
        "D2 0-13 CASO": data[(only_pareados) & (d2_013) & (tipo_caso)],
        "D2 0-13 CONTROLE": data[(only_pareados) & (d2_013) & (tipo_controle)],
        "D2 >=14 TOTAL": data[(only_pareados) & (d2_14)],
        "D2 >=14 CASO": data[(only_pareados) & (d2_14) & (tipo_caso)],
        "D2 >=14 CONTROLE": data[(only_pareados) & (d2_14) & (tipo_controle)],
    }
    return hashd

# VE general - CORONAVAC

## Prepare configuration vars and Load data

In [4]:
config = {
    "init cohort": dt.date(2021, 1, 21),
    "final cohort": dt.date(2021, 8, 31),
    "seeds": np.arange(1,16,1),
    "data folder": None,
    "file_pareados": lambda seed: f"pareados_corona_{seed}.csv",
    "file_eventos": lambda seed: f"pares_eventos_corona_{seed}.csv",
    "pop_cohort": "pop_reservoir_corona.csv"
}
str_init = f"{config['init cohort'].day}{config['init cohort'].strftime('%b').upper()}{config['init cohort'].year}"
str_final = f"{config['final cohort'].day}{config['final cohort'].strftime('%b').upper()}{config['final cohort'].year}"
config["data folder"] = os.path.join("..", "output", "data", "COHORT_"+str_init+"_"+str_final)

In [5]:
# EVENTOS
seed = 1
df_eventos_pares = pd.read_csv(os.path.join(config["data folder"], config["file_eventos"](seed)))
df_eventos_pares.info()

FileNotFoundError: [Errno 2] No such file or directory: '..\\output\\data\\COHORT_21JAN2021_31AUG2021\\pares_eventos_corona_1.csv'

In [4]:
df_eventos_pares.describe()

,CPF CASO,CPF CONTROLE,INTV OBITO COVID CASO(D1),INTV OBITO GERAL CASO(D1),INTV OBITO COVID CONTROLE(D1),INTV OBITO GERAL CONTROLE(D1),INTV D1 CASO CONTROLE(D1),INTV FIM COORTE(D1),INTV OBITO COVID CASO(D2),INTV OBITO GERAL CASO(D2),INTV OBITO COVID CONTROLE(D2),INTV OBITO GERAL CONTROLE(D2),INTV D1 CASO CONTROLE(D2),INTV FIM COORTE(D2)
count,1.103890e+05,1.103890e+05,248.000000,610.000000,672.000000,1081.000000,87743.000000,110389.000000,71.000000,503.000000,657.000000,1072.000000,86656.000000,108769.000000
mean,2.516625e+10,2.736228e+10,41.274194,100.793443,36.270833,94.177613,8.881096,160.203788,44.718310,87.328032,7.774734,68.205224,-20.668875,130.650581
std,2.369156e+10,2.451608e+10,28.483347,54.401686,27.963376,57.313497,17.531579,13.967337,34.687661,48.818080,31.886199,59.566680,21.054848,20.179040
min,2.233010e+05,2.233010e+05,-20.000000,-155.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-81.000000,-70.000000,-154.000000,-1.000000
25%,7.418744e+09,9.127283e+09,23.000000,53.000000,17.000000,45.000000,1.000000,152.000000,18.000000,43.500000,-14.000000,19.750000,-33.000000,118.000000
50%,1.549939e+10,1.681122e+10,36.000000,103.000000,29.000000,91.000000,3.000000,158.000000,38.000000,91.000000,1.000000,66.500000,-24.000000,129.000000
75%,3.801128e+10,4.168687e+10,52.000000,144.000000,48.000000,144.000000,7.000000,162.000000,54.500000,126.000000,23.000000,117.000000,-14.000000,138.000000
max,9.999750e+10,9.999719e+10,147.000000,221.000000,161.000000,230.000000,214.000000,222.000000,133.000000,191.000000,143.000000,220.000000,194.000000,200.000000


In [6]:
# PAREADOS
df_pareados = pd.read_csv(os.path.join(config["data folder"], config["file_pareados"](seed)))
df_pareados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229437 entries, 0 to 229436
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   CPF               229437 non-null  int64  
 1   DATA D1           204965 non-null  object 
 2   DATA D2           200579 non-null  object 
 3   DATA OBITO COVID  1096 non-null    object 
 4   DATA OBITO GERAL  1853 non-null    object 
 5   TIPO              229437 non-null  object 
 6   PAR               220898 non-null  float64
 7   PAREADO           229437 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 12.5+ MB


In [7]:
df_pareados.describe()

,CPF,PAR
count,2.294370e+05,2.208980e+05
mean,2.631574e+10,2.625689e+10
std,2.411122e+10,2.413617e+10
min,2.233010e+05,2.233010e+05
25%,8.848963e+09,8.617454e+09
50%,1.642840e+10,1.638530e+10
75%,3.913253e+10,3.912093e+10
max,9.999750e+10,9.999750e+10


In [8]:
# POP COHORT
df_pop = pd.read_csv(os.path.join(config["data folder"], config["pop_cohort"]))
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187921 entries, 0 to 187920
Data columns (total 41 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   nome                                   187921 non-null  object 
 1   cpf                                    187921 non-null  int64  
 2   data_nascimento                        187921 non-null  object 
 3   bairro                                 187921 non-null  object 
 4   sexo                                   187921 non-null  object 
 5   created_at                             187921 non-null  object 
 6   nome_mae                               187921 non-null  object 
 7   data D1(VACINADOS)                     163449 non-null  object 
 8   data D2(VACINADOS)                     159767 non-null  object 
 9   vacina(VACINADOS)                      163449 non-null  object 
 10  fornecedor(VACINADOS)                  163445 non-null  

In [9]:
df_pop.describe()

,cpf,idade anos(VACINADOS),bairro id(VACINADOS),do_8,do_8(CARTORIOS),idade
count,1.879210e+05,163449.000000,143531.000000,9.620000e+02,2.454000e+03,187921.000000
mean,2.646710e+10,70.247337,73.293156,3.004125e+07,3.080349e+07,70.415552
std,2.417828e+10,6.303299,46.233427,2.532544e+06,2.207954e+06,6.577705
min,2.233010e+05,6.000000,1.000000,2.541104e+07,2.541103e+07,60.000000
25%,8.988439e+09,65.000000,34.000000,3.106031e+07,3.139295e+07,65.000000
50%,1.650324e+10,69.000000,71.000000,3.140046e+07,3.148084e+07,69.000000
75%,3.932256e+10,73.000000,105.000000,3.148524e+07,3.149357e+07,74.000000
max,9.999750e+10,140.000000,158.000000,3.201800e+07,3.290127e+07,230.000000


In [10]:
datasets = {
    "EVENTOS": None,
    "PARES": df_pareados,
    "POP": df_pop,
    "EVENTO_INFO": None
}

df_evento_info = surv.push_info(datasets["EVENTOS"], datasets["POP"])
datasets["EVENTO_INFO"] = df_evento_info
df_evento_info.info()

AttributeError: 'NoneType' object has no attribute 'merge'

## Counting

In [11]:
def stratify_pares(data, data_pop):
    '''
    
    '''
    # --> collect data on age and sex
    data = data.merge(data_pop[["cpf", "sexo", "idade", "bairro"]], left_on="CPF", right_on="cpf", how="left")
    cols = ["DATA D1", "DATA D2"]
    for j in cols:
        data[j] = pd.to_datetime(data[j], format="%Y-%m-%d", errors="coerce")
    
    data["VACINA STATUS ATE FIM DA COORTE"] = data.apply(lambda x: f_vacina(x, config["final cohort"]), axis=1)
    
    # --> conditions
    only_pareados = data["PAREADO"]==True
    only_d1 = pd.notna(data["DATA D1"]) & pd.isna(data["DATA D2"])
    d1_and_d2 = pd.notna(data["DATA D1"]) & pd.notna(data["DATA D2"])
    no_vaccine = pd.isna(data["DATA D1"]) & pd.isna(data["DATA D2"])
    males = data["sexo"]=="M"
    females = data["sexo"]=="F"
    idade_6069 = (data["idade"]>=60) & (data["idade"]<=69)
    idade_7079 = (data["idade"]>=70) & (data["idade"]<=79)
    idade_80 = (data["idade"]>=80) & (data["idade"]<=105)
    tipo_caso = data["TIPO"]=="CASO"
    tipo_controle = data["TIPO"]=="CONTROLE"
    obito_covid = pd.notna(data["DATA OBITO COVID"])
    obito_geral = pd.notna(data["DATA OBITO GERAL"])
    novac = data["VACINA STATUS ATE FIM DA COORTE"]=="NO VACCINE"
    d1_013 = data["VACINA STATUS ATE FIM DA COORTE"]=="D1 0-13 DAYS"
    d1_14 = data["VACINA STATUS ATE FIM DA COORTE"]=="D1 >= 14 DAYS"
    d2_013 = data["VACINA STATUS ATE FIM DA COORTE"]=="D2 0-13 DAYS"
    d2_14 = data["VACINA STATUS ATE FIM DA COORTE"]=="D2 >= 14 DAYS"
    
    hashd = {
        "TOTAL": data[only_pareados],
        "CASO": data[only_pareados & tipo_caso],
        "CONTROLE": data[only_pareados & tipo_controle],
        "NO VACCINE": data[(only_pareados) & (no_vaccine)],
        "ONLY D1": data[(only_pareados) & (only_d1)],
        "D1+D2": data[(only_pareados) & (d1_and_d2)],
        "MALES": data[(only_pareados) & (males)],
        "MALES CASO": data[(only_pareados) & (males) & (tipo_caso)],
        "MALES CONTROLE": data[(only_pareados) & (males) & (tipo_controle)],
        "FEMALES": data[(only_pareados) & (females)],
        "FEMALES CASO": data[(only_pareados) & (females) & (tipo_caso)],
        "FEMALES CONTROLE": data[(only_pareados) & (females) & (tipo_controle)],
        "60-69": data[(only_pareados) & (idade_6069)],
        "60-69 CASO": data[(only_pareados) & (idade_6069) & (tipo_caso)],
        "60-69 CONTROLE": data[(only_pareados) & (idade_6069) & (tipo_controle)],
        "70-79": data[(only_pareados) & (idade_7079)],
        "70-79 CASO": data[(only_pareados) & (idade_7079) & (tipo_caso)],
        "70-79 CONTROLE": data[(only_pareados) & (idade_7079) & (tipo_controle)],
        "80+": data[(only_pareados) & (idade_80)],
        "80+ CASO": data[(only_pareados) & (idade_80) & (tipo_caso)],
        "80+ CONTROLE": data[(only_pareados) & (idade_80) & (tipo_controle)],
        "OBITO COVID TOTAL": data[(only_pareados) & (obito_covid)],
        "OBITO COVID CASO": data[(only_pareados) & (obito_covid) & (tipo_caso)],
        "OBITO COVID CONTROLE": data[(only_pareados) & (obito_covid) & (tipo_controle)],
        "OBITO GERAL TOTAL": data[(only_pareados) & (obito_geral)],
        "OBITO GERAL CASO": data[(only_pareados) & (obito_geral) & (tipo_caso)],
        "OBITO GERAL CONTROLE": data[(only_pareados) & (obito_geral) & (tipo_controle)],
        "SEM VACINA TOTAL": data[(only_pareados) & (novac)],
        "SEM VACINA CASO": data[(only_pareados) & (novac) & (tipo_caso)],
        "SEM VACINA CONTROLE": data[(only_pareados) & (novac) & (tipo_controle)],
        "D1 0-13 TOTAL": data[(only_pareados) & (d1_013)],
        "D1 0-13 CASO": data[(only_pareados) & (d1_013) & (tipo_caso)],
        "D1 0-13 CONTROLE": data[(only_pareados) & (d1_013) & (tipo_controle)],
        "D1 >=14 TOTAL": data[(only_pareados) & (d1_14)],
        "D1 >=14 CASO": data[(only_pareados) & (d1_14) & (tipo_caso)],
        "D1 >=14 CONTROLE": data[(only_pareados) & (d1_14) & (tipo_controle)],
        "D2 0-13 TOTAL": data[(only_pareados) & (d2_013)],
        "D2 0-13 CASO": data[(only_pareados) & (d2_013) & (tipo_caso)],
        "D2 0-13 CONTROLE": data[(only_pareados) & (d2_013) & (tipo_controle)],
        "D2 >=14 TOTAL": data[(only_pareados) & (d2_14)],
        "D2 >=14 CASO": data[(only_pareados) & (d2_14) & (tipo_caso)],
        "D2 >=14 CONTROLE": data[(only_pareados) & (d2_14) & (tipo_controle)],
    }
    return hashd

def f_vacina(x, final_cohort):
    '''
        Define vaccination status based only on the cohort without outcome.
    '''
    dist_days = None
    if pd.isna(x["DATA D1"]) and pd.isna(x["DATA D2"]):
        dist_days = np.nan
        return "NO VACCINE"
    elif not pd.isna(x["DATA D2"]):
        dist_days = (final_cohort - x["DATA D2"].date()).days
        if dist_days<14:
            return "D2 0-13 DAYS"
        else:
            return "D2 >= 14 DAYS"
    elif not pd.isna(x["DATA D1"]):
        dist_days = (final_cohort - x["DATA D1"].date()).days
        if dist_days<14:
            return "D1 0-13 DAYS"
        else:
            return "D1 >= 14 DAYS"

In [12]:
hash_data = stratify_pares(datasets["PARES"], datasets["POP"])

In [13]:
cols = {"TOTAL": None, "CASO": None, "CONTROLE": None}
rows = {"Mean age(SD)": dict(cols), "Age 60-69": dict(cols), 
        "Age 70-79": dict(cols), "Age 80+": dict(cols), "Male": dict(cols), 
        "Female": dict(cols), "Death Covid": dict(cols), "Death NOT Covid": dict(cols), 
        "Vac -> Not Vaccinated": dict(cols), "Vac -> D1 0-13 DAYS": dict(cols), "Vac -> D1 >= 14 DAYS": dict(cols), 
        "Vac -> D1+D2 0-13 DAYS": dict(cols), "Vac -> D1+D2 >= 14 DAYS": dict(cols), "Total": dict(cols)}

In [14]:
def fill_table(rows, hash_data):
    rows["Total"]["TOTAL"] = f'{hash_data["TOTAL"].shape[0]}'
    rows["Total"]["CASO"] = f'{hash_data["CASO"].shape[0]}'
    rows["Total"]["CONTROLE"] = f'{hash_data["CONTROLE"].shape[0]}'
    rows["Mean age(SD)"]["TOTAL"] = f'{hash_data["TOTAL"]["idade"].mean():.2f}({hash_data["TOTAL"]["idade"].std():.2f})'
    rows["Mean age(SD)"]["CASO"] = f'{hash_data["CASO"]["idade"].mean():.2f}({hash_data["CASO"]["idade"].std():.2f})'
    rows["Mean age(SD)"]["CONTROLE"] = f'{hash_data["CONTROLE"]["idade"].mean():.2f}({hash_data["CONTROLE"]["idade"].std():.2f})'
    rows["Age 60-69"]["TOTAL"] = hash_data["60-69"].shape[0]
    rows["Age 60-69"]["CASO"] =  hash_data["60-69 CASO"].shape[0]
    rows["Age 60-69"]["CONTROLE"] = hash_data["60-69 CONTROLE"].shape[0]
    rows["Age 70-79"]["TOTAL"] = hash_data["70-79"].shape[0]
    rows["Age 70-79"]["CASO"] = hash_data["70-79 CASO"].shape[0]
    rows["Age 70-79"]["CONTROLE"] = hash_data["70-79 CONTROLE"].shape[0]
    rows["Age 80+"]["TOTAL"] = hash_data["80+"].shape[0]
    rows["Age 80+"]["CASO"] = hash_data["80+ CASO"].shape[0]
    rows["Age 80+"]["CONTROLE"] = hash_data["80+ CONTROLE"].shape[0]
    rows["Male"]["TOTAL"] = hash_data["MALES"].shape[0]
    rows["Male"]["CASO"] = hash_data["MALES CASO"].shape[0]
    rows["Male"]["CONTROLE"] = hash_data["MALES CONTROLE"].shape[0]
    rows["Female"]["TOTAL"] = hash_data["FEMALES"].shape[0]
    rows["Female"]["CASO"] = hash_data["FEMALES CASO"].shape[0]
    rows["Female"]["CONTROLE"] = hash_data["FEMALES CONTROLE"].shape[0]
    rows["Death Covid"]["TOTAL"] = hash_data["OBITO COVID TOTAL"].shape[0]
    rows["Death Covid"]["CASO"] = hash_data["OBITO COVID CASO"].shape[0]
    rows["Death Covid"]["CONTROLE"] = hash_data["OBITO COVID CONTROLE"].shape[0]
    rows["Death NOT Covid"]["TOTAL"] = hash_data["OBITO GERAL TOTAL"].shape[0]
    rows["Death NOT Covid"]["CASO"] = hash_data["OBITO GERAL CASO"].shape[0]
    rows["Death NOT Covid"]["CONTROLE"] = hash_data["OBITO GERAL CONTROLE"].shape[0]
    rows["Vac -> Not Vaccinated"]["TOTAL"] = hash_data["SEM VACINA TOTAL"].shape[0]
    rows["Vac -> Not Vaccinated"]["CASO"] = hash_data["SEM VACINA CASO"].shape[0]
    rows["Vac -> Not Vaccinated"]["CONTROLE"] = hash_data["SEM VACINA CONTROLE"].shape[0]
    rows["Vac -> D1 0-13 DAYS"]["TOTAL"] = hash_data["D1 0-13 TOTAL"].shape[0]
    rows["Vac -> D1 0-13 DAYS"]["CASO"] = hash_data["D1 0-13 CASO"].shape[0]
    rows["Vac -> D1 0-13 DAYS"]["CONTROLE"] = hash_data["D1 0-13 CONTROLE"].shape[0]
    rows["Vac -> D1 >= 14 DAYS"]["TOTAL"] = hash_data["D1 >=14 TOTAL"].shape[0]
    rows["Vac -> D1 >= 14 DAYS"]["CASO"] = hash_data["D1 >=14 CASO"].shape[0]
    rows["Vac -> D1 >= 14 DAYS"]["CONTROLE"] = hash_data["D1 >=14 CONTROLE"].shape[0]
    rows["Vac -> D1+D2 0-13 DAYS"]["TOTAL"] = hash_data["D2 0-13 TOTAL"].shape[0]
    rows["Vac -> D1+D2 0-13 DAYS"]["CASO"] = hash_data["D2 0-13 CASO"].shape[0]
    rows["Vac -> D1+D2 0-13 DAYS"]["CONTROLE"] = hash_data["D2 0-13 CONTROLE"].shape[0]
    rows["Vac -> D1+D2 >= 14 DAYS"]["TOTAL"] = hash_data["D2 >=14 TOTAL"].shape[0]
    rows["Vac -> D1+D2 >= 14 DAYS"]["CASO"] = hash_data["D2 >=14 CASO"].shape[0]
    rows["Vac -> D1+D2 >= 14 DAYS"]["CONTROLE"] = hash_data["D2 >=14 CONTROLE"].shape[0]
    
    return pd.DataFrame(rows).T

In [15]:
fill_table(rows, hash_data)

,TOTAL,CASO,CONTROLE
Mean age(SD),70.44(6.17),70.92(6.14),69.96(6.16)
Age 60-69,110653,52450,58203
Age 70-79,90552,47459,43093
Age 80+,19674,10530,9144
Male,85474,42737,42737
Female,135424,67712,67712
Death Covid,919,244,675
Death NOT Covid,1690,612,1078
Vac -> Not Vaccinated,22647,0,22647
Vac -> D1 0-13 DAYS,352,81,271


In [203]:
def count_deaths(hash_data):
    rows = {
        "SEM VACINA TOTAL": None, 
        "D1 0-13 TOTAL": None,
        "D1 >=14 TOTAL": None,
        "D2 0-13 TOTAL": None,
        "D2 >=14 TOTAL": None
    }
    
    for key in hash_data.keys():
        hash_data[key]["OBITO POR COVID"] = hash_data[key]["DATA OBITO COVID"].apply(lambda x: "NAO" if pd.isna(x) else "SIM")
    
    for key in rows.keys():
        rows[key] = pd.crosstab(hash_data[key]["TIPO"], hash_data[key]["OBITO POR COVID"])
    return rows
    
    

In [205]:
cross = count_deaths(hash_data)

In [218]:
cross["D2 >=14 TOTAL"].T

TIPO,CASO,CONTROLE
OBITO POR COVID,,
NAO,108620,84821
SIM,71,63


In [216]:
(71/108620)/(63/84821)

0.8800581903417477

In [219]:
63+71

134